In [2]:
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()

with conn:
    c.execute(
        """
    create table if not exists stat (id integer primary key, cat text, score real);
    """
    )

    c.execute("""insert into stat (cat, score) values ('a', 1.0);""")
    c.execute("""insert into stat (cat, score) values ('b', 2.0);""")
    c.execute("""insert into stat (cat, score) values ('c', 4.0);""")
    c.execute("""insert into stat (cat, score) values ('d', 6.0);""")
    
    result = c.execute("""select * from stat;""").fetchall()

    print(result)

[(1, 'a', 1.0), (2, 'b', 2.0), (3, 'c', 4.0), (4, 'd', 6.0)]


In [3]:
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()

with conn:
    c.execute(
        """
    create table if not exists
        stat (id integer primary key, cat text, score real);
    """
    )

    # Data needs to be passed as an iterable of tuples.
    data = (
        ("a", 1.0),
        ("b", 2.0),
        ("c", 3.0), ("d", 4.0), ("e", 5.0), ("f", 6.0), ("g", 7.0), 
        ("h", 8.0), ("i", 9.0))

    c.executemany("insert into stat (cat, score) values (?, ?);", data)
    result = c.execute("""select * from stat;""").fetchall()

    print(result)

[(1, 'a', 1.0), (2, 'b', 2.0), (3, 'c', 3.0), (4, 'd', 4.0), (5, 'e', 5.0), (6, 'f', 6.0), (7, 'g', 7.0), (8, 'h', 8.0), (9, 'i', 9.0)]


### Applying user-defined aggregate functions

In [6]:
import sqlite3
import hashlib

conn = sqlite3.connect(":memory:")
c = conn.cursor()

class Mult:
    def __init__(self):
        self._result = 1

    def step(self, value):
        self._result *= value

    def finalize(self):
        return self._result

# Register the aggregate class.
conn.create_aggregate("mult", 1, Mult)

with conn:
    c.execute(
        """
        create table if not exists series (
            val integer
        );
    """
    )
    c.execute("insert into series (val) values (?);", (1,))
    c.execute("insert into series (val) values (?);", (4,))
    c.execute("insert into series (val) values (?);", (5,))
    c.execute("insert into series (val) values (?);", (6,))

    result = c.execute("select mult(val) from series;").fetchall()
    print(result)

[(120,)]


In [7]:
import datetime
import sqlite3
import zoneinfo

conn = sqlite3.connect(
    ":memory:",
    detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
)

with conn:
    c = conn.cursor()
    c.execute(
        """
        create table if not exists
        timekeeper (id integer primary key, d date, dt timestamp);"""
    )
    tz = zoneinfo.ZoneInfo("Europe/Madrid")
    dt = datetime.datetime.now(tz)
    d = dt.date()

    c.execute(
        "insert into timekeeper (d, dt) values (?, ?);",
        ((d, dt)),
    )
    result = c.execute(
        """
        select
            d as "d [date]", dt as "dt [timestamp]"
            from timekeeper;"""
    ).fetchall()
    print(result)

[(datetime.date(2023, 2, 17), datetime.datetime(2023, 2, 17, 20, 8, 59, 898131))]


In [9]:
import sqlite3

conn = sqlite3.connect(":memory:")

# Using a dictionary to represent a row.
def row_factory(cursor, row):
    # cursor.description:
    # (name, type_code, display_size,
    # internal_size, precision, scale, null_ok)
    # row: (value, value, ...)
    return {
        col[0]: row[idx]
        for idx, col in enumerate(
            cursor.description,)}

conn.row_factory = row_factory

c = conn.cursor()

with conn:
    c.execute(
        """
        create table if not exists colors (
            name text,
            hex text);""")
    c.execute(
        "insert into colors (name, hex) values (?, ?);",
        ("red", "#ff0000"),)
    c.execute(
        "insert into colors (name, hex) values (?, ?);",
        ("green", "#00ff00"),)
    c.execute(
        "insert into colors (name, hex) values (?, ?);",
        ("blue", "#0000ff"),)

    result = c.execute("select * from colors;").fetchall()
    print(result)

[{'name': 'red', 'hex': '#ff0000'}, {'name': 'green', 'hex': '#00ff00'}, {'name': 'blue', 'hex': '#0000ff'}]


In [10]:
import sqlite3

conn = sqlite3.connect(":memory:")
# Registering a highly optimized 'Row' object as the
# default row_factory. Row is a map-like object that
# allows you to access column values by name.
conn.row_factory = sqlite3.Row

c = conn.cursor()

with conn:
    c.execute(
        """
        create table if not exists colors (
            name text,
            hex text);""")
    c.executemany(
        "insert into colors (name, hex) values (?, ?);",
        (
            ("red", "#ff0000"),
            ("green", "#00ff00"),
            ("blue", "#0000ff"),
        ),)

    result = c.execute("select * from colors;").fetchall()
    # Access the values of a row by column name.
    for row in result:
        print(row["name"], row["hex"])
    # Convert the result to a list of dicts.
    result_dict = [dict(row) for row in result]
    print(result_dict)

red #ff0000
green #00ff00
blue #0000ff
[{'name': 'red', 'hex': '#ff0000'}, {'name': 'green', 'hex': '#00ff00'}, {'name': 'blue', 'hex': '#0000ff'}]


In [2]:
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
# Print all the statements executed.
def introspect(s):
    print(s)

# Register the trace function.
conn.set_trace_callback(introspect)

with conn:
    c.execute("create table if not exists colors (name text);")
    c.executemany(
        "insert into colors (name) values (?);",
        (("red",), ("green",), ("blue",)),)

    result = c.execute("""select * from colors""").fetchall()
    print(result)

conn.close()

create table if not exists colors (name text);
BEGIN 
insert into colors (name) values ('red');
insert into colors (name) values ('green');
insert into colors (name) values ('blue');
select * from colors
[('red',), ('green',), ('blue',)]
COMMIT
